In [234]:
import json

import altair as alt

alt.data_transformers.disable_max_rows()
import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from folium import FeatureGroup
from folium.plugins import TimeSliderChoropleth
from matplotlib import cm
from matplotlib.colors import Normalize, to_hex
from shapely.geometry import mapping

In [235]:
# Load the data
years = list(range(1980, 2025))
dfs = []

# Assuming all the CSV files are in the format 'daily_aqi_by_county_<year>.csv'
for year in years:
    file_name = f"daily_aqi_data/daily_aqi_by_county_{year}.csv"
    df = pd.read_csv(file_name)
    dfs.append(df)

data = pd.concat(dfs, ignore_index=True)

# Ensure 'Date' column is in datetime format
data["Date"] = pd.to_datetime(data["Date"])
data["Year"] = data["Date"].dt.year

state_yearly_aqi = data.groupby(["State Name", "Year"])["AQI"].mean().reset_index()

/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_37846/819555585.py:8: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_37846/819555585.py:8: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_37846/819555585.py:8: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_37846/819555585.py:8: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_37846/819555585.py:8: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_37846/819555585.py:8: Dtyp

In [236]:
population_data = pd.read_csv("us_population_data_1910_2020.csv")


merged_data = pd.merge(
    state_yearly_aqi,
    population_data[
        ["Name", "Year", "Resident Population", "Resident Population Density"]
    ],
    left_on=["State Name", "Year"],
    right_on=["Name", "Year"],
    how="inner",
)

# Drop duplicate 'Name' column (if needed)
merged_data = merged_data.drop(columns=["Name"])

merged_data["Resident Population Density"] = pd.to_numeric(
    merged_data["Resident Population Density"], errors="coerce"
).fillna(0)

merged_data["Resident Population"] = (
    merged_data["Resident Population"].astype(str).str.replace(",", "").astype(float)
)

merged_data["logpop"] = np.log(merged_data["Resident Population"])

merged_data["AQI"] = pd.to_numeric(merged_data["AQI"], errors="coerce")

merged_data = merged_data[merged_data["State Name"] != "Puerto Rico"]

merged_data

,State Name,Year,AQI,Resident Population,Resident Population Density,logpop
0,Alabama,1980,58.110709,3893888.0,76.9,15.174919
1,Alabama,1990,46.135135,4040587.0,79.8,15.211901
2,Alabama,2000,64.233499,4447100.0,87.8,15.307763
3,Alabama,2010,52.630990,4779736.0,94.4,15.379896
4,Alabama,2020,41.556068,5024279.0,99.2,15.429793
...,...,...,...,...,...,...
247,Wisconsin,2020,41.136414,5893718.0,108.8,15.589398
248,Wyoming,1990,36.620581,453588.0,4.7,13.024945
249,Wyoming,2000,37.185334,493782.0,5.1,13.109849
250,Wyoming,2010,39.553744,563626.0,5.8,13.242146


In [312]:
# Plot population density vs AQI over the years
state_to_region = {
    "Connecticut": "New England",
    "Maine": "New England",
    "Massachusetts": "New England",
    "New Hampshire": "New England",
    "Rhode Island": "New England",
    "Vermont": "New England",
    "Illinois": "Midwest",
    "Indiana": "Midwest",
    "Iowa": "Midwest",
    "Kansas": "Midwest",
    "Michigan": "Midwest",
    "Minnesota": "Midwest",
    "Missouri": "Midwest",
    "Nebraska": "Midwest",
    "New York": "Mid-Atlantic",
    "New Jersey": "Mid-Atlantic",
    "North Dakota": "Midwest",
    "Ohio": "Midwest",
    "Pennsylvania": "Mid-Atlantic",
    "South Dakota": "Midwest",
    "Wisconsin": "Midwest",
    "Delaware": "South",
    "Florida": "South",
    "Georgia": "South",
    "Maryland": "South",
    "North Carolina": "South",
    "South Carolina": "South",
    "Virginia": "South",
    "District of Columbia": "South",
    "West Virginia": "South",
    "Alabama": "South",
    "Kentucky": "South",
    "Mississippi": "South",
    "Tennessee": "South",
    "Arkansas": "South",
    "Louisiana": "South",
    "Oklahoma": "South",
    "Texas": "South",
    "Arizona": "West",
    "Colorado": "West",
    "Idaho": "West",
    "Montana": "West",
    "Nevada": "West",
    "New Mexico": "West",
    "Utah": "West",
    "Wyoming": "West",
    "Alaska": "West",
    "California": "West",
    "Hawaii": "West",
    "Oregon": "West",
    "Washington": "West",
}

merged_data["Region"] = merged_data["State Name"].map(state_to_region)

year_slider = alt.binding_range(
    min=merged_data["Year"].min(), max=merged_data["Year"].max(), step=10, name="Year:"
)
year_select = alt.selection_point(fields=["Year"], bind=year_slider, name="Year")

region_radio = alt.binding_radio(
    options=["New England", "Midwest", "Mid-Atlantic", "South", "West", "All"],
    name="Region:",
    labels=["New England", "Midwest", "Mid-Atlantic", "South", "West", "All"],
)

region_param = alt.param(
    name="Region",
    bind=region_radio,
    value="All",
)

scatter_plot = (
    alt.Chart(merged_data)
    .mark_circle(size=100)
    .encode(
        x=alt.X("Resident Population Density:Q", title="Resident Population Density"),
        y=alt.Y("AQI:Q", title="Air Quality Index"),
        color=alt.condition(
            'datum.Region == Region || Region == "All"',
            alt.Color("Region:N", legend=alt.Legend(title="Region")),
            alt.value("lightgrey"),
        ),
        tooltip=["State Name", "Resident Population Density", "AQI", "Year", "Region"],
    )
    .add_params(year_select, region_param)
    .transform_filter(year_select)
    .properties(
        width=600,
        height=400,
        title="Population Density vs Air Quality Index (AQI) by Year",
    )
)

scatter_plot

alt.Chart(...)

In [ ]:
geojson_path = "us-states.json"

# Load GeoJSON file as a dictionary
with open(geojson_path, "r") as f:
    geojson_data = json.load(f)

# Normalize data for consistent color scaling
pop_norm = Normalize(vmin=merged_data["logpop"].min(), vmax=merged_data["logpop"].max())
aqi_norm = Normalize(vmin=merged_data["AQI"].min(), vmax=merged_data["AQI"].max())

geojson = {"type": "FeatureCollection", "features": []}

# Iterate over the states in the original GeoJSON
for feature in geojson_data["features"]:
    state_name = feature["properties"]["name"]
    matching_rows = merged_data[merged_data["State Name"] == state_name]

    # Add a feature for each year for this state
    for _, row in matching_rows.iterrows():
        new_feature = {
            "type": "Feature",
            "geometry": feature["geometry"],
            "properties": {
                "name": state_name,
                "year": row["Year"],
                "Resident Population": row["Resident Population"],
                "Resident Population Density": row["Resident Population Density"],
                "AQI": row["AQI"],
            },
        }
        geojson["features"].append(new_feature)

# Prepare styledict dynamically based on Resident Population Density
styledict = {}

for feature in geojson["features"]:
    state_name = feature["properties"]["name"]
    year = feature["properties"]["year"]
    density = feature["properties"]["Resident Population Density"]

    if state_name not in styledict:
        styledict[state_name] = {}

    styledict[state_name][year] = {
        "color": to_hex(cm.Blues(pop_norm(density))),
        "opacity": 0.7,
    }

# Initialize the Folium map
folium_map = folium.Map(location=[37.8, -96], zoom_start=4)

# Add Resident Population Choropleth
population_choropleth = folium.Choropleth(
    geo_data=geojson,
    data=merged_data[merged_data["Year"] == merged_data["Year"].max()],
    columns=["State Name", "Resident Population"],
    key_on="feature.properties.name",
    fill_color="Greys",
    fill_opacity=0.3,
    line_opacity=0.2,
    legend_name="Resident Population",
    name="Resident Population",
).add_to(folium_map)

# Add Tooltip to Resident Population Choropleth
folium.GeoJsonTooltip(
    fields=["name", "Resident Population", "Resident Population Density", "AQI"],
    aliases=["State:", "Population:", "Population Density:", "AQI:"],
    localize=True,
).add_to(population_choropleth.geojson)

# Add AQI Choropleth
aqi_choropleth = folium.Choropleth(
    geo_data=geojson,
    data=merged_data[merged_data["Year"] == merged_data["Year"].max()],
    columns=["State Name", "AQI"],
    key_on="feature.properties.name",
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Average AQI",
    name="AQI",
).add_to(folium_map)

# Add Tooltip to AQI Choropleth
folium.GeoJsonTooltip(
    fields=["name", "Resident Population", "Resident Population Density", "AQI"],
    aliases=["State:", "Population:", "Population Density:", "AQI:"],
    localize=True,
).add_to(aqi_choropleth.geojson)

# Add GeoJSON with tooltips for enriched data
folium.GeoJson(
    geojson,
    tooltip=folium.GeoJsonTooltip(
        fields=["name", "Resident Population", "Resident Population Density", "AQI"],
        aliases=["State:", "Population:", "Population Density: ", "AQI:"],
        localize=True,
    ),
    style_function=lambda feature: {
        "fillColor": "white",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.1,
        "opacity": 0.8,
    },
    name="Population Tooltip",
    control=False,
).add_to(folium_map)

folium.LayerControl().add_to(folium_map)

folium_map